Code for figuring out the object detection using open-cv

In [1]:
import cv2
import apriltag
import numpy as np
import time

In [2]:
def detect_puck_position(frame, lower_color, upper_color):
    if frame is None:
        print("Error: Invalid frame.")
        return None, None
    
    mask = cv2.inRange(frame, lower_color, upper_color)
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # assume largest contour is the puck
        largest_contour = max(contours, key=cv2.contourArea)
        
        if cv2.contourArea(largest_contour) > 100:  # Filter out noise
            (x, y), radius = cv2.minEnclosingCircle(largest_contour)
            center = (int(x), int(y))
            
            # Draw a point at the center of the puck
            cv2.circle(frame, center, 25, (0, 255, 0), -1)
            return center, frame
    
    return None, frame


In [3]:
def main():
    #black puck range
    lower_color = np.array([0, 0, 0])
    upper_color = np.array([100, 100, 100])
    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam. Try changing the device index.")
        return
    
    print("Starting puck detection. Press 'q' to quit.")

    
    fps_start_time = time.time()
    time_start = fps_start_time
    fps_frame_count = 0
    fps = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Can't receive frame. Exiting...")
            break
        
        puck_position, processed_frame = detect_puck_position(frame, lower_color, upper_color)
        
        # Calculate and display FPS
        fps_frame_count += 1
        if (time.time() - fps_start_time) > 1:
            fps = fps_frame_count / (time.time() - fps_start_time)
            fps_frame_count = 0
            fps_start_time = time.time()
        
        # Display position information on the frame
        if puck_position:
            position_text = f"Position: {puck_position}, Time: {time.time() - time_start:.2f}"
            print(position_text)
            cv2.putText(processed_frame, position_text, (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            cv2.putText(processed_frame, "No puck detected", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        cv2.putText(processed_frame, f"FPS: {fps:.1f}", (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        
        # Display the mask to help with debugging color thresholds
        mask = cv2.inRange(frame, lower_color, upper_color)
        cv2.imshow("Mask", mask)
            
        # Display the resulting frame
        cv2.imshow("Puck Detection", processed_frame)
        
        # Break the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # When everything is done, release the capture
    if cap.isOpened():
        cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == "__main__":
    main()


2025-04-10 15:50:41.732 Python[95830:3536968] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Starting puck detection. Press 'q' to quit.
Position: (449, 393), Time: 0.32
Position: (449, 389), Time: 0.47
Position: (451, 388), Time: 0.54
Position: (450, 388), Time: 0.64
Position: (450, 387), Time: 0.71
Position: (451, 387), Time: 0.79
Position: (450, 389), Time: 0.88
Position: (451, 390), Time: 0.94
Position: (451, 392), Time: 0.99


2025-04-10 15:50:43.734 Python[95830:3536968] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-10 15:50:43.734 Python[95830:3536968] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Position: (452, 392), Time: 1.07
Position: (452, 390), Time: 1.15
Position: (458, 390), Time: 1.23
Position: (639, 349), Time: 1.32
Position: (639, 223), Time: 1.38
Position: (639, 206), Time: 1.42
Position: (639, 258), Time: 1.47
Position: (639, 403), Time: 1.53
Position: (639, 443), Time: 1.58
Position: (639, 433), Time: 1.67
Position: (686, 449), Time: 1.77
Position: (694, 458), Time: 1.84
Position: (724, 479), Time: 1.92
Position: (639, 479), Time: 2.10
Position: (639, 466), Time: 2.14
Position: (639, 91), Time: 2.17
Position: (639, 103), Time: 2.22
Position: (639, 428), Time: 2.29
Position: (639, 402), Time: 2.36
Position: (639, 351), Time: 2.42
Position: (639, 347), Time: 2.45
Position: (639, 335), Time: 2.50
Position: (642, 383), Time: 2.54
Position: (641, 476), Time: 2.60
Position: (639, 504), Time: 2.66
Position: (639, 595), Time: 2.71
Position: (372, 830), Time: 2.77
Position: (353, 907), Time: 2.81
Position: (98, 236), Time: 2.85
Position: (133, 202), Time: 2.89
Position: (2

In [4]:
def detect_apriltags(frame):
    """Detect AprilTags in a frame and return their positions and IDs."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Create an AprilTag detector
    options = apriltag.DetectorOptions(families="tag36h11")
    detector = apriltag.Detector(options)
    
    # Detect tags
    results = detector.detect(gray)
    
    tags = []
    for r in results:
        tag_id = r.tag_id
        center = (int(r.center[0]), int(r.center[1]))
        corners = np.array(r.corners, dtype=np.int32).reshape((-1, 1, 2))
        
        tags.append({
            'id': tag_id,
            'center': center,
            'corners': corners
        })
        
        # Draw tag outline and ID on the frame
        cv2.polylines(frame, [corners], True, (0, 255, 0), 2)
        cv2.putText(frame, str(tag_id), center, cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7, (0, 0, 255), 2)
        
    return tags, frame

def calibrate_goal():
    """
    Calibrate the camera to detect the goal position using AprilTags.
    Place AprilTag ID 0 on the left goalpost and ID 1 on the right goalpost.
    """
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam. Try changing the device index.")
        return None
    
    print("Starting goal calibration.")
    print("Place AprilTag ID 0 on the left goalpost and ID 1 on the right goalpost.")
    print("Press 'c' to capture the calibration once both tags are visible.")
    print("Press 'q' to quit without calibrating.")
    
    goal_posts = {'left': None, 'right': None}
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Can't receive frame. Exiting...")
            break
        
        tags, frame_with_tags = detect_apriltags(frame)
        
        # Check if both tags are visible
        for tag in tags:
            if tag['id'] == 0:  # Left post tag
                goal_posts['left'] = tag['center']
                cv2.putText(frame_with_tags, "Left post detected", 
                           (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            if tag['id'] == 1:  # Right post tag
                goal_posts['right'] = tag['center']
                cv2.putText(frame_with_tags, "Right post detected", 
                           (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Display instructions
        cv2.putText(frame_with_tags, "Position tags and press 'c' to calibrate", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Draw goal line if both posts are detected
        if goal_posts['left'] and goal_posts['right']:
            cv2.line(frame_with_tags, goal_posts['left'], goal_posts['right'], 
                    (0, 255, 255), 2)
        
        cv2.imshow("Goal Calibration", frame_with_tags)
        
        # Handle key presses
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            goal_posts = {'left': None, 'right': None}
            break
        elif key == ord('c') and goal_posts['left'] and goal_posts['right']:
            print("Goal calibrated successfully!")
            print(f"Left post at: {goal_posts['left']}")
            print(f"Right post at: {goal_posts['right']}")
            break
    
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)  # Ensure all windows close properly
    
    return goal_posts

def draw_goal_on_frame(frame, goal_posts):
    """Draw the calibrated goal on a frame for visualization"""
    if goal_posts['left'] and goal_posts['right']:
        # Draw goal posts
        cv2.circle(frame, goal_posts['left'], 10, (0, 0, 255), -1)
        cv2.circle(frame, goal_posts['right'], 10, (0, 0, 255), -1)
        
        # Draw goal line
        cv2.line(frame, goal_posts['left'], goal_posts['right'], (0, 255, 255), 2)
        
        cv2.putText(frame, "Goal Calibrated", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    else:
        cv2.putText(frame, "Goal Not Calibrated", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    return frame

# Call this function to run the calibration
# goal_posts = calibrate_goal()